# Segmenting and Clustering Neighborhoods in Toronto

First we import pandas and request
Then we get the data from URL and we remove the 'Not assigned' rows

In [144]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

dfr = pd.read_html(response.content, header=0)[0]
dfx = dfr[dfr.Borough != 'Not assigned']

dfx.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Then we search for rows with the same postcode and we concatenate the strings of Neighbourhood.
Then we create a new table starting from the old one and we drop duplicates.

In [146]:

df2 = dfx
label = ''
df3 = pd.DataFrame({'Postcode': [], 'Borough': [], 'Neighbourhood': []})
for index, row in df2.iterrows():
    label = ''
    ct = df2.loc[df2['Postcode'] == row["Postcode"]]
    

    #print(ct)
    i = 0
    for index, row in ct.iterrows():
        i += 1
        if (i > 1): label = label + ', '
        label = label +row['Neighbourhood']
#Postcode:  ct.iloc[0]['Postcode']
#Borough:  ct.iloc[0]['Borough']
#Neighbourhood: label
    df3 = df3.append({'Postcode': ct.iloc[0]['Postcode'], 'Borough': ct.iloc[0]['Borough'], 'Neighbourhood': label}, ignore_index=True)

df3.drop_duplicates(subset=None, keep='first', inplace=True)


df3.head()

,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
4,North York,"Lawrence Heights, Lawrence Manor",M6A
6,Queen's Park,Not assigned,M7A


If we have Neighbourhood Not assigned, we change it with the value of Borough

In [149]:
import numpy as np
df3['Neighbourhood'] = np.where(df3['Neighbourhood']== 'Not assigned', df3['Borough'], df3['Neighbourhood'])
df3.head()


,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
4,North York,"Lawrence Heights, Lawrence Manor",M6A
6,Queen's Park,Queen's Park,M7A


We use the shape function to display rows and columns numbers

In [153]:

print("We have %s columns" % (df3.shape[1]))
print("We have %s rows" % (df3.shape[0]))

We have 3 columns
We have 103 rows


We read the coordinates and then we inner join our dataframe with the coordinates

In [159]:
dfcoordinates = pd.read_csv("https://cocl.us/Geospatial_data")
df3 = df3.join(dfcoordinates.set_index('Postal Code'), on='Postcode')
df3.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
4,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
6,Queen's Park,Queen's Park,M7A,43.662301,-79.389494
